Ⅰ数据预处理
    1.将已经完成基本处理的训练集数据文件导入，进行异常值检查和清理。
        分类变量的处理有：
        (1)环线变量：按照2环、4环、6环划分，进行数值编码1、2、3、4代表离中心的远近。
        (2)房屋户型：按照有几室进行数值编码，12-16室的数据极少（<0.1%）且与其他数据相差太大，进行删除处理。
        (3)所在楼层：按照地下室、低层、中层和高层进行划分，数值编码1、2、3、4，表示高度。
        (4)房屋朝向：按照是否为南北朝向划分为二元变量。
        (5)装修情况：按照未知、毛胚、简装、精装进行分类为0、1、2、3代表装修的完善程度。
        (6)配备电梯：按照是否配备电梯划分为二元变量。
        (7)别墅类型：按照非别墅、叠拼、联排、独栋划分为数值0、1、2、3，代表房屋的独立性程度。
        (8)交易权属：是否为商品房划分0-1变量。
        (9)房屋用途：是否为住宅（非商用）划分0-1变量。
        (10)房屋年限：未知（0），未满两年（1），满两年（2），满五年（5）.
        (11)产权所属：是否共有划分为0-1变量。
        (12)周边配备：是否有配备介绍划分为0-1变量。
        (13)交通出行：是否有地铁公交描述划分为0-1变量。
    2.进行异常值检验、日期识别等数据处理
Ⅱ ols模型
    1.ols模型的训练
    2.ols模型的预测
    3.6折交叉验证
Ⅲ lasso模型
    1.lasso模型的训练
    2.lasso模型的预测
    3.6折交叉验证
Ⅳ ridge模型
    1.ridge模型的训练
    2.ridge模型的预测
    3.6折交叉验证

In [311]:
##数据读取

import pandas as pd
file_path = "C:\\Users\\HP\\Desktop\\mid_project\\train.csv"
# 尝试不同编码格式
try:
    data = pd.read_csv(file_path, encoding="utf-8")  # UTF-8 编码
except UnicodeDecodeError:
    try:
        data = pd.read_csv(file_path, encoding="gbk")  # GBK 编码（适用于中文）
    except UnicodeDecodeError:
        data = pd.read_csv(file_path, encoding="latin1")  # Latin-1 编码（适用于部分 ANSI 文件）
print("数据读取成功！")

# 交易时间和建筑面积的数据提取
data['交易时间'] = pd.to_datetime(data['交易时间'], format='%Y/%m/%d')  # 转换为日期格式
data['交易时间'] = data['交易时间'].dt.year  # 提取年份
# 去掉 "㎡" 并转换为 float 类型
data['建筑面积'] = data['建筑面积'].str.replace('㎡', '', regex=False).astype(float)



数据读取成功！


In [313]:
#异常值处理

import numpy as np

# 计算建筑面积的 IQR 并去除异常值
Q1_area = data['建筑面积'].quantile(0.25)
Q3_area = data['建筑面积'].quantile(0.75)
IQR_area = Q3_area - Q1_area
lower_bound_area = Q1_area - 1.5 * IQR_area
upper_bound_area = Q3_area + 1.5 * IQR_area

# 计算房价（单位面积价格）的 IQR 并去除异常值
Q1_price = data['价格'].quantile(0.25)
Q3_price = data['价格'].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

# 过滤数据
filtered_data = data[
    (data['建筑面积'] >= lower_bound_area) & (data['建筑面积'] <= upper_bound_area) & 
    (data['价格'] >= lower_bound_price) & (data['价格'] <= upper_bound_price)
]
print(f"去除异常值前的数据量: {data.shape[0]}")
print(f"去除异常值后的数据量: {filtered_data.shape[0]}")

data=filtered_data

去除异常值前的数据量: 84124
去除异常值后的数据量: 74588


In [315]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

#变量准备
df = pd.DataFrame(data)
df['价格'] = pd.to_numeric(df['价格'], errors='coerce')
df['建筑面积'] = pd.to_numeric(df['建筑面积'], errors='coerce')
# 计算建筑单价（价格 / 建筑面积）
df['建筑单价'] = df['价格'] / df['建筑面积']
# 对价格进行对数变换
df['log_建筑单价'] = np.log(df['建筑单价'])
# 对建筑面积进行对数变换，+1是为了避免建筑面积为零时的计算问题
df['log_建筑面积'] = np.log(df['建筑面积']+1 )  # 加1避免零值

# 使用get_dummies对城市进行独热编码
df_city_encoded = pd.get_dummies(df['城市'], prefix='城市')
# 将独热编码的结果与原数据合并
df = pd.concat([df, df_city_encoded], axis=1)
df.drop('城市', axis=1, inplace=True)

# 将城市_0到城市_6的所有列转换为数值类型（0 和 1）
city_columns = ['城市_0', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6']
df[city_columns] = df[city_columns].astype(int)
print(df.head())


   区域   板块  环线       价格  房屋户型  所在楼层    建筑面积  房屋朝向  装修情况  配备电梯  ...  \
1  43  231   4  4174000     3     4  127.44     1     3     0  ...   
3  62  568   3  2834600     2     2   43.60     1     3     1  ...   
4  62  226   3  1954000     1     3   39.85     1     3     1  ...   
5  45  616   3  2250000     1     3   69.30     1     3     1  ...   
8  45  559   4  3138000     2     4   90.60     1     3     0  ...   

           建筑单价   log_建筑单价  log_建筑面积  城市_0  城市_1  城市_2  城市_3  城市_4  城市_5  城市_6  
1  32752.667922  10.396740  4.855462     1     0     0     0     0     0     0  
3  65013.761468  11.082354  3.797734     1     0     0     0     0     0     0  
4  49033.877039  10.800267  3.709907     1     0     0     0     0     0     0  
5  32467.532468  10.387996  4.252772     1     0     0     0     0     0     0  
8  34635.761589  10.452642  4.517431     1     0     0     0     0     0     0  

[5 rows x 29 columns]


In [ ]:
Ⅱ ols模型

In [319]:
#模型训练
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# 选择特征变量和目标变量
X = df[['板块', '区域', '环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况', 'log_建筑面积', '别墅类型', '交易时间', '房屋用途', '房屋年限',   
        '交通出行', '年份'] + list(df_city_encoded.columns)]  
y = df['log_建筑单价']

# 1. 先划分数据（保持 80% 训练, 20% 测试）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

# 2.目标编码
# 计算训练集中的区域均值
region_mean_map = X_train.join(y_train).groupby('区域')['log_建筑单价'].mean()
# 用训练集均值填充训练集
X_train['区域_encoded'] = X_train['区域'].map(region_mean_map)
# 在测试集上进行编码时，只使用训练集信息
X_test['区域_encoded'] = X_test['区域'].map(region_mean_map)
#X_test['区域_encoded'] = X_test['区域_encoded'].fillna(region_mean_map.mean())  # 使用训练集的均值填充

# 3. 删除原始的'区域'列
X_train.drop(columns=['区域'], inplace=True)
X_test.drop(columns=['区域'], inplace=True)

# 4. 创建交互项（Interaction Terms）

X_train['环线_交通出行_interaction'] = X_train['环线'] * X_train['交通出行']
X_test['环线_交通出行_interaction'] = X_test['环线'] * X_test['交通出行']

X_train['环线_房屋用途_interaction'] = X_train['环线'] * X_train['房屋用途']
X_test['环线_房屋用途_interaction'] = X_test['环线'] * X_test['房屋用途']

X_train['装修情况_房屋用途_interaction'] = X_train['装修情况'] * X_train['房屋用途']
X_test['装修情况_房屋用途_interaction'] = X_test['装修情况'] * X_test['房屋用途']

# 5. 变量名处理（避免 statsmodels 误修改）避免空格
X_train.columns = [col.replace(' ', '_') for col in X_train.columns]  # 确保列名合法
X_test.columns = [col.replace(' ', '_') for col in X_test.columns]

# 6. 添加常数项
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# 7. 训练 OLS 回归模型
model = sm.OLS(y_train, X_train)
results = model.fit()

# 8. 输出回归结果
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:               log_建筑单价   R-squared:                       0.827
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                 1.239e+04
Date:                Thu, 03 Apr 2025   Prob (F-statistic):               0.00
Time:                        16:37:39   Log-Likelihood:                -5005.7
No. Observations:               59670   AIC:                         1.006e+04
Df Residuals:                   59646   BIC:                         1.028e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -20.15

In [321]:
#模型评估1：rmse与mae
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# 1. 预测 log_建筑单价
X_test = sm.add_constant(X_test)  # 确保测试集有常数项
X_train = sm.add_constant(X_train)  # 确保训练集有常数项

y_pred_log_train = results.predict(X_train)  # 训练集预测
y_pred_log_test = results.predict(X_test)  # 测试集预测

# 2. 反变换得到预测建筑单价
y_pred_price_per_sqm_train = np.exp(y_pred_log_train)  
y_pred_price_per_sqm_test = np.exp(y_pred_log_test)  

# 3. 计算真实价格和预测价格
df['建筑面积'] = pd.to_numeric(df['建筑面积'], errors='coerce')  # 确保建筑面积是数值
train_data = df.loc[y_train.index]
test_data = df.loc[y_test.index]

y_train_price = np.exp(y_train) * train_data['建筑面积']  # 训练集真实房产价格
y_test_price = np.exp(y_test) * test_data['建筑面积']  # 测试集真实房产价格

y_pred_price_train = y_pred_price_per_sqm_train * train_data['建筑面积']  # 训练集预测房产价格
y_pred_price_test = y_pred_price_per_sqm_test * test_data['建筑面积']  # 测试集预测房产价格

# 4. 处理NaN值
valid_train_indices = (~y_train_price.isna()) & (~y_pred_price_train.isna())
valid_test_indices = (~y_test_price.isna()) & (~y_pred_price_test.isna())

y_train_price = y_train_price[valid_train_indices]
y_pred_price_train = y_pred_price_train[valid_train_indices]

y_test_price = y_test_price[valid_test_indices]
y_pred_price_test = y_pred_price_test[valid_test_indices]

# 5. 计算 RMSE 和 MAE
rmse_train = root_mean_squared_error(y_train_price, y_pred_price_train)
mae_train = mean_absolute_error(y_train_price, y_pred_price_train)

rmse_test = root_mean_squared_error(y_test_price, y_pred_price_test)
mae_test = mean_absolute_error(y_test_price, y_pred_price_test)

print(f'训练集 房产价格的 RMSE: {rmse_train:.2f}')
print(f'训练集 房产价格的 MAE: {mae_train:.2f}')
print(f'测试集 房产价格的 RMSE: {rmse_test:.2f}')
print(f'测试集 房产价格的 MAE: {mae_test:.2f}')


训练集 房产价格的 RMSE: 392060.50
训练集 房产价格的 MAE: 251145.69
测试集 房产价格的 RMSE: 392291.13
测试集 房产价格的 MAE: 253947.38


In [323]:
#模型预测
import pandas as pd

# 指定文件路径
file_path = r"C:\Users\HP\Desktop\mid_project\test_.xlsx"

# 导入Excel文件
test_data = pd.read_excel(file_path)
test_data['交易时间'] = pd.to_datetime(test_data['交易时间'], format='%Y/%m/%d')  # 转换为日期格式
test_data['交易时间'] = test_data['交易时间'].dt.year  # 提取年份
test_data['建筑面积'] = test_data['建筑面积'].str.replace('㎡', '', regex=False).astype(float)
df_test = pd.DataFrame(test_data)
df_test['log_建筑面积'] = np.log(df_test['建筑面积']+1 )  # 加1避免零值
# 查看计算结果
print(df_test[['ID','建筑面积', 'log_建筑面积']].head())


# 使用get_dummies对城市进行独热编码
df_test_city_encoded = pd.get_dummies(df_test['城市'], prefix='城市')
df_test = pd.concat([df_test, df_test_city_encoded], axis=1)
# 删除原来的'城市'列
df_test.drop('城市', axis=1, inplace=True)

# 将城市_0到城市_6的所有列转换为数值类型（0 和 1）
city_columns = ['城市_0', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6']
df_test[city_columns] = df_test[city_columns].astype(int)

# 打印合并后的数据集查看效果
print(df.head())

   ID    建筑面积  log_建筑面积
0   0  209.20  5.348059
1   1  163.69  5.104065
2   2  102.92  4.643621
3   3  109.66  4.706462
4   4   57.20  4.063885
   区域   板块  环线       价格  房屋户型  所在楼层    建筑面积  房屋朝向  装修情况  配备电梯  ...  \
1  43  231   4  4174000     3     4  127.44     1     3     0  ...   
3  62  568   3  2834600     2     2   43.60     1     3     1  ...   
4  62  226   3  1954000     1     3   39.85     1     3     1  ...   
5  45  616   3  2250000     1     3   69.30     1     3     1  ...   
8  45  559   4  3138000     2     4   90.60     1     3     0  ...   

           建筑单价   log_建筑单价  log_建筑面积  城市_0  城市_1  城市_2  城市_3  城市_4  城市_5  城市_6  
1  32752.667922  10.396740  4.855462     1     0     0     0     0     0     0  
3  65013.761468  11.082354  3.797734     1     0     0     0     0     0     0  
4  49033.877039  10.800267  3.709907     1     0     0     0     0     0     0  
5  32467.532468  10.387996  4.252772     1     0     0     0     0     0     0  
8  34635.761589  10.452642  4.

In [325]:
# 1. 确保 df_test_data 是独立副本，防止 SettingWithCopyWarning
X_test_data = df_test[['板块', '区域', '环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况', 'log_建筑面积', '别墅类型', '交易时间', '房屋用途', '房屋年限',   
                       '交通出行', '年份'] + list(df_test_city_encoded.columns)].copy()  # **加 copy()**

# 2. 目标编码（避免数据泄露）
X_test_data.loc[:, '区域_encoded'] = X_test_data['区域'].map(region_mean_map)
X_test_data['区域_encoded'] = X_test_data['区域_encoded'].fillna(region_mean_map.mean())

# 3. 删除原始的“区域”列
X_test_data.drop(columns=['区域'], inplace=True)

X_test_data['环线_交通出行_interaction'] = X_test_data['环线'] * X_test_data['交通出行']

X_test_data['环线_房屋用途_interaction'] = X_test_data['环线'] * X_test_data['房屋用途']

X_test_data['装修情况_房屋用途_interaction'] = X_test_data['装修情况'] * X_test_data['房屋用途']


# 4. 确保变量名一致
X_test_data.columns = [col.replace(' ', '_') for col in X_test_data.columns]

# 5. 添加常数项
X_test_data = sm.add_constant(X_test_data)

# 6. 预测建筑单价
predictions_log = results.predict(X_test_data)
predictions = np.exp(predictions_log)  # 反变换

# 7. 把预测结果加回 df_test
df_test['预测建筑单价'] = predictions
df_test['区域_encoded'] = X_test_data['区域_encoded']  # **确保区域编码加回 df_test**

# 8. 计算价格 = 预测建筑单价 * 建筑面积
df_test['建筑面积'] = pd.to_numeric(df_test['建筑面积'], errors='coerce')  # 确保建筑面积是数值型
df_test['预测价格'] = df_test['预测建筑单价'] * df_test['建筑面积']

# 9. 查看最终结果
print(df_test[['区域_encoded', '预测建筑单价', '建筑面积', '预测价格']].head())


   区域_encoded        预测建筑单价    建筑面积          预测价格
0   10.705832  46552.381740  209.20  9.738758e+06
1   10.705832  51219.708256  163.69  8.384154e+06
2   10.445561  33503.618749  102.92  3.448192e+06
3   10.018846  21314.465996  109.66  2.337344e+06
4   11.165736  76479.925456   57.20  4.374652e+06


In [327]:
# 假设 'ID' 是你的预测集中的一个列名，且 '预测价格' 已经计算得出
output_df = df_test[['ID', '预测价格']]  # 提取 ID 和 预测价格

# 保存为 CSV 文件
output_path = r"C:\Users\HP\Desktop\mid_project\预测结果_ols.csv"
output_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"预测结果已保存到 {output_path}")


预测结果已保存到 C:\Users\HP\Desktop\mid_project\预测结果_ols.csv


In [ ]:
Ⅱ lasso模型

In [335]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 确保 X, y 定义正确
X = df[['板块', '区域', '环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况', 'log_建筑面积', 
        '别墅类型', '交易时间', '房屋用途', '房屋年限', '交通出行', '年份'] + list(df_city_encoded.columns)]  
y = df['log_建筑单价']  # 目标变量
building_area = np.exp(df['log_建筑面积'])  # 还原建筑面积

# 1. 设置6折交叉验证
kf = KFold(n_splits=6, shuffle=True, random_state=111)

rmse_list = []
mae_list = []

for train_index, val_index in kf.split(X):
    # **划分训练集和验证集**
    X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
    y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()

    # **获取验证集的建筑面积（确保是原始面积）**
    area_val = np.exp(df.loc[X_val.index, 'log_建筑面积'])   

    # 目标编码（区域变量）
    region_mean_map = X_train.join(y_train).groupby('区域')['log_建筑单价'].mean()
    X_train['区域_encoded'] = X_train['区域'].map(region_mean_map)
    X_val['区域_encoded'] = X_val['区域'].map(region_mean_map)

# 处理缺失值，避免警告
    X_val['区域_encoded'] = X_val['区域_encoded'].fillna(y_train.mean())

    # 删除原始的 '区域' 列
    X_train.drop(columns=['区域'], inplace=True)
    X_val.drop(columns=['区域'], inplace=True)

    # 交互特征
    X_train['环线_交通出行_interaction'] = X_train['环线'] * X_train['交通出行']
    X_val['环线_交通出行_interaction'] = X_val['环线'] * X_val['交通出行']

    X_train['环线_房屋用途_interaction'] = X_train['环线'] * X_train['房屋用途']
    X_val['环线_房屋用途_interaction'] = X_val['环线'] * X_val['房屋用途']

    X_train['装修情况_房屋用途_interaction'] = X_train['装修情况'] * X_train['房屋用途']
    X_val['装修情况_房屋用途_interaction'] = X_val['装修情况'] * X_val['房屋用途']

    # **5. 确保列名合法**
    X_train.columns = [col.replace(' ', '_') for col in X_train.columns]
    X_val.columns = [col.replace(' ', '_') for col in X_val.columns]

    # **6. 添加常数项**
    X_train = sm.add_constant(X_train)
    X_val = sm.add_constant(X_val)

    # **7. 训练回归模型**
    model = sm.OLS(y_train, X_train)
    results = model.fit()

    # **8. 预测并还原 log**
    y_val_pred_log = results.predict(X_val)
    y_val_pred_unit_price = np.exp(y_val_pred_log)  # 还原 log 单价
    y_val_actual_unit_price = np.exp(y_val)  # 还原 log 单价

    # **9. 计算价格**
    y_val_pred_price = y_val_pred_unit_price * area_val  # 预测价格
    y_val_actual_price = y_val_actual_unit_price * area_val  # 真实价格

    # **10. 计算 RMSE 和 MAE**
    rmse = np.sqrt(mean_squared_error(y_val_actual_price, y_val_pred_price))
    mae = mean_absolute_error(y_val_actual_price, y_val_pred_price)

    rmse_list.append(rmse)
    mae_list.append(mae)

# **11. 输出 6 折交叉验证的平均 RMSE 和 MAE**
print(f"Cross-validated RMSE (价格): {np.mean(rmse_list):.4f}")
print(f"Cross-validated MAE (价格): {np.mean(mae_list):.4f}")


Cross-validated RMSE (价格): 397301.3650
Cross-validated MAE (价格): 255007.1672


In [342]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# 假设数据已加载为 DataFrame df
file_path = "C:\\Users\\HP\\Desktop\\mid_project\\train.csv"

# 尝试不同编码格式
try:
    data = pd.read_csv(file_path, encoding="utf-8")  # UTF-8 编码
except UnicodeDecodeError:
    try:
        data = pd.read_csv(file_path, encoding="gbk")  # GBK 编码（适用于中文）
    except UnicodeDecodeError:
        data = pd.read_csv(file_path, encoding="latin1")  # Latin-1 编码（适用于部分 ANSI 文件）

print("数据读取成功！")

# 数据预处理
data['交易时间'] = pd.to_datetime(data['交易时间'], format='%Y/%m/%d')  # 转换为日期格式
data['交易时间'] = data['交易时间'].dt.year  # 提取年份
data['建筑面积'] = data['建筑面积'].str.replace('㎡', '', regex=False).astype(float)


# 对建筑面积和价格进行对数变换
data['建筑单价'] = data['价格'] / data['建筑面积']
data['log_建筑单价'] = np.log(data['建筑单价'])
data['log_建筑面积'] = np.log(data['建筑面积'] + 1)  # 防止零值

# 独热编码
df_city_encoded = pd.get_dummies(data['城市'], prefix='城市')
data = pd.concat([data, df_city_encoded], axis=1)
data.drop('城市', axis=1, inplace=True)

# 选择特征和目标变量
X = data[[ '板块', '区域','环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况',  '交易权属','log_建筑面积',
        '配备电梯', '别墅类型', '交易时间', '房屋用途', '房屋年限', '产权所属', '周边配套', 
        '交通出行', '年份'] + list(df_city_encoded.columns)]
y = data['log_建筑单价']

# **数据划分**
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

# **目标编码**
region_mean_map = X_train.join(y_train).groupby('区域')['log_建筑单价'].mean()
X_train['区域_encoded'] = X_train['区域'].map(region_mean_map)
X_test['区域_encoded'] = X_test['区域'].map(region_mean_map)
#X_test['区域_encoded'] = X_test['区域_encoded'].fillna(y_train.mean())


# 删除原始“区域”列
X_train.drop(columns=['区域'], inplace=True)
X_test.drop(columns=['区域'], inplace=True)

# 标准化数据
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lasso回归
lasso_model = Lasso(alpha=0.001)  # alpha 是Lasso的正则化参数，你可以调整这个值
lasso_model.fit(X_train_scaled, y_train)

# 预测
y_pred = lasso_model.predict(X_test_scaled)

# 计算均方误差（MSE）
mse = mean_squared_error(y_test, y_pred)
print(f"Lasso回归均方误差: {mse}")

# 打印回归系数
print(f"Lasso回归系数: {lasso_model.coef_}")

# 反变换到原始价格（预测的对数价格反变换）
predictions = np.exp(y_pred)  # 反变换log(建筑单价)



数据读取成功！
Lasso回归均方误差: 0.07711216682767362
Lasso回归系数: [ 2.33343829e-03  1.26777576e-02  4.46899339e-02 -8.38694122e-03
  4.00430815e-03  2.16224817e-02  2.58378224e-02 -9.39638393e-03
  2.59167008e-02  1.77578065e-02  3.93480239e-03  6.40889910e-02
 -2.05946412e-02 -5.88006305e-04  6.05261538e-04  8.91790988e-03
  5.64323466e-03  5.08249370e-03  1.09784591e-02 -1.36362175e-02
 -3.61809812e-03 -3.67871038e-03  6.00850922e-03 -0.00000000e+00
  7.16445622e-01]


In [345]:

df_test = pd.read_excel(r"C:\Users\HP\Desktop\mid_project\test_.xlsx")
df_test['交易时间'] = pd.to_datetime(df_test['交易时间'], format='%Y/%m/%d')
df_test['交易时间'] = df_test['交易时间'].dt.year
df_test['建筑面积'] = df_test['建筑面积'].str.replace('㎡', '', regex=False).astype(float)

# 对建筑面积进行对数变换
df_test['log_建筑面积'] = np.log(df_test['建筑面积'] + 1)

# 独热编码
df_test_city_encoded = pd.get_dummies(df_test['城市'], prefix='城市')
df_test = pd.concat([df_test, df_test_city_encoded], axis=1)
df_test.drop('城市', axis=1, inplace=True)

X_test_data = df_test[[ '板块', '区域','环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况',  '交易权属','log_建筑面积',
        '配备电梯', '别墅类型', '交易时间', '房屋用途', '房屋年限', '产权所属', '周边配套', 
        '交通出行', '年份']
 + list(df_test_city_encoded.columns)].copy()

X_test_data['区域_encoded'] = X_test_data['区域'].map(region_mean_map)
X_test_data['区域_encoded'] = X_test_data['区域_encoded'].fillna(y_train.mean())


# 删除原始“区域”列
X_test_data.drop(columns=['区域'], inplace=True)

# 标准化测试集数据
X_test_data_scaled = scaler.transform(X_test_data)

# Lasso预测
predictions_test = lasso_model.predict(X_test_data_scaled)

# 反变换到原始价格
predictions_test = np.exp(predictions_test)

# 加入到df_test
df_test['预测建筑单价'] = predictions_test
df_test['建筑面积'] = pd.to_numeric(df_test['建筑面积'], errors='coerce')
df_test['预测价格'] = df_test['预测建筑单价'] * df_test['建筑面积']

# 输出结果
output_df = df_test[['ID', '预测价格']]
output_path = r"C:\Users\HP\Desktop\mid_project\预测结果_lasso_04.csv"
output_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"预测结果已保存到 {output_path}")


预测结果已保存到 C:\Users\HP\Desktop\mid_project\预测结果_lasso_04.csv


In [348]:
from sklearn.metrics import mean_absolute_error

# 训练集预测
y_train_pred = lasso_model.predict(X_train_scaled)

# 测试集预测
y_test_pred = lasso_model.predict(X_test_scaled)

# **还原 log(建筑单价) 到原始价格**
# 计算真实价格（反变换）
y_train_real = np.exp(y_train) * data.loc[X_train.index, '建筑面积']
y_test_real = np.exp(y_test) * data.loc[X_test.index, '建筑面积']

# 计算预测价格（反变换）
y_train_pred_real = np.exp(y_train_pred) * data.loc[X_train.index, '建筑面积']
y_test_pred_real = np.exp(y_test_pred) * data.loc[X_test.index, '建筑面积']

# **计算 RMSE 和 MAE**
rmse_train = np.sqrt(mean_squared_error(y_train_real, y_train_pred_real))
rmse_test = np.sqrt(mean_squared_error(y_test_real, y_test_pred_real))

mae_train = mean_absolute_error(y_train_real, y_train_pred_real)
mae_test = mean_absolute_error(y_test_real, y_test_pred_real)

# **打印结果**
print(f"训练集 RMSE: {rmse_train:.2f}, MAE: {mae_train:.2f}")
print(f"测试集 RMSE: {rmse_test:.2f}, MAE: {mae_test:.2f}")


训练集 RMSE: 1078078.99, MAE: 430336.42
测试集 RMSE: 1036909.44, MAE: 426132.62


In [351]:
# 定义 6 折交叉验证
kf = KFold(n_splits=6, shuffle=True, random_state=42)

# 存储评估指标
rmse_scores = []
mae_scores = []

for train_index, val_index in kf.split(X):
    # **划分训练集和验证集**
    X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
    y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()

    # **获取验证集的建筑面积（确保是原始面积）**
    area_val = np.exp(data.loc[X_val.index, 'log_建筑面积'])  

    # 目标编码（区域变量）
    region_mean_map = X_train.join(y_train).groupby('区域')['log_建筑单价'].mean()
    X_train['区域_encoded'] = X_train['区域'].map(region_mean_map)
    X_val['区域_encoded'] = X_val['区域'].map(region_mean_map)

# 处理缺失值，避免警告
    X_val['区域_encoded'] = X_val['区域_encoded'].fillna(y_train.mean())


    # **标准化**
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # **训练 Lasso 回归**
    lasso_model = Lasso(alpha=0.001)
    lasso_model.fit(X_train_scaled, y_train)

    # **预测**
    y_val_pred = lasso_model.predict(X_val_scaled)

    # **反变换 + 乘以建筑面积计算总价**
    y_val_pred_price = np.exp(y_val_pred) * area_val  # 预测总价
    y_val_actual_price = np.exp(y_val) * area_val  # 真实总价

    # **计算 RMSE 和 MAE**
    rmse = np.sqrt(mean_squared_error(y_val_actual_price, y_val_pred_price))
    mae = mean_absolute_error(y_val_actual_price, y_val_pred_price)

    rmse_scores.append(rmse)
    mae_scores.append(mae)

# **计算交叉验证的平均 RMSE 和 MAE**
mean_rmse = np.mean(rmse_scores)
mean_mae = np.mean(mae_scores)

# **打印结果**
print(f"6折交叉验证 平均 RMSE（总价）: {mean_rmse:.2f}")
print(f"6折交叉验证 平均 MAE（总价）: {mean_mae:.2f}")

6折交叉验证 平均 RMSE（总价）: 1076150.30
6折交叉验证 平均 MAE（总价）: 434193.59


In [ ]:
Ⅲ ridge

In [354]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge  # 用 Ridge 替代 Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# 假设数据已加载为 DataFrame df
file_path = "C:\\Users\\HP\\Desktop\\mid_project\\train.csv"

# 尝试不同编码格式
try:
    data = pd.read_csv(file_path, encoding="utf-8")  # UTF-8 编码
except UnicodeDecodeError:
    try:
        data = pd.read_csv(file_path, encoding="gbk")  # GBK 编码（适用于中文）
    except UnicodeDecodeError:
        data = pd.read_csv(file_path, encoding="latin1")  # Latin-1 编码（适用于部分 ANSI 文件）

print("数据读取成功！")

# 数据预处理
data['交易时间'] = pd.to_datetime(data['交易时间'], format='%Y/%m/%d')  # 转换为日期格式
data['交易时间'] = data['交易时间'].dt.year  # 提取年份
data['建筑面积'] = data['建筑面积'].str.replace('㎡', '', regex=False).astype(float)


# 对建筑面积和价格进行对数变换
data['建筑单价'] = data['价格'] / data['建筑面积']
data['log_建筑单价'] = np.log(data['建筑单价'])
data['log_建筑面积'] = np.log(data['建筑面积'] + 1)  # 防止零值

# 独热编码
df_city_encoded = pd.get_dummies(data['城市'], prefix='城市')
data = pd.concat([data, df_city_encoded], axis=1)
data.drop('城市', axis=1, inplace=True)

# 选择特征和目标变量
X = data[[ '板块', '区域','环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况',  '交易权属','log_建筑面积',
        '配备电梯', '别墅类型', '交易时间', '房屋用途', '房屋年限', '产权所属', '周边配套', 
        '交通出行', '年份'] + list(df_city_encoded.columns)]
y = data['log_建筑单价']

# **数据划分**
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

# **目标编码**
region_mean_map = X_train.join(y_train).groupby('区域')['log_建筑单价'].mean()
X_train['区域_encoded'] = X_train['区域'].map(region_mean_map)
X_test['区域_encoded'] = X_test['区域'].map(region_mean_map)
X_test['区域_encoded'] = X_test['区域_encoded'].fillna(y_train.mean())


# 删除原始“区域”列
X_train.drop(columns=['区域'], inplace=True)
X_test.drop(columns=['区域'], inplace=True)

# 标准化数据
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 岭回归
ridge_model = Ridge(alpha=1.0)  # alpha 是岭回归的正则化参数，你可以调整这个值
ridge_model.fit(X_train_scaled, y_train)

# 预测
y_pred = ridge_model.predict(X_test_scaled)

# 计算均方误差（MSE）
mse = mean_squared_error(y_test, y_pred)
print(f"Ridge回归均方误差: {mse}")

# 打印回归系数
print(f"Ridge回归系数: {ridge_model.coef_}")

# 反变换到原始价格（预测的对数价格反变换）
predictions = np.exp(y_pred)  # 反变换log(建筑单价)

# 预测文件加入
df_test = pd.read_excel(r"C:\Users\HP\Desktop\mid_project\test_.xlsx")
df_test['交易时间'] = pd.to_datetime(df_test['交易时间'], format='%Y/%m/%d')
df_test['交易时间'] = df_test['交易时间'].dt.year
df_test['建筑面积'] = df_test['建筑面积'].str.replace('㎡', '', regex=False).astype(float)

# 对建筑面积进行对数变换
df_test['log_建筑面积'] = np.log(df_test['建筑面积'] + 1)

# 独热编码
df_test_city_encoded = pd.get_dummies(df_test['城市'], prefix='城市')
df_test = pd.concat([df_test, df_test_city_encoded], axis=1)
df_test.drop('城市', axis=1, inplace=True)

X_test_data = df_test[[ '板块', '区域','环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况',  '交易权属','log_建筑面积',
        '配备电梯', '别墅类型', '交易时间', '房屋用途', '房屋年限', '产权所属', '周边配套', 
        '交通出行', '年份'] + list(df_test_city_encoded.columns)].copy()

X_test_data['区域_encoded'] = X_test_data['区域'].map(region_mean_map)
X_test_data['区域_encoded'] = X_test_data['区域_encoded'].fillna(y_train.mean())


# 删除原始“区域”列
X_test_data.drop(columns=['区域'], inplace=True)

# 标准化测试集数据
X_test_data_scaled = scaler.transform(X_test_data)

# Ridge预测
predictions_test = ridge_model.predict(X_test_data_scaled)

# 反变换到原始价格
predictions_test = np.exp(predictions_test)

# 加入到df_test
df_test['预测建筑单价'] = predictions_test
df_test['建筑面积'] = pd.to_numeric(df_test['建筑面积'], errors='coerce')
df_test['预测价格'] = df_test['预测建筑单价'] * df_test['建筑面积']

# 输出结果
output_df = df_test[['ID', '预测价格']]
output_path = r"C:\Users\HP\Desktop\mid_project\预测结果_ridge_04.csv"
output_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"预测结果已保存到 {output_path}")

# 训练集预测
y_train_pred = ridge_model.predict(X_train_scaled)

# 测试集预测
y_test_pred = ridge_model.predict(X_test_scaled)

# **还原 log(建筑单价) 到原始价格**
# 计算真实价格（反变换）
y_train_real = np.exp(y_train) * data.loc[X_train.index, '建筑面积']
y_test_real = np.exp(y_test) * data.loc[X_test.index, '建筑面积']

# 计算预测价格（反变换）
y_train_pred_real = np.exp(y_train_pred) * data.loc[X_train.index, '建筑面积']
y_test_pred_real = np.exp(y_test_pred) * data.loc[X_test.index, '建筑面积']

# **计算 RMSE 和 MAE**
rmse_train = np.sqrt(mean_squared_error(y_train_real, y_train_pred_real))
rmse_test = np.sqrt(mean_squared_error(y_test_real, y_test_pred_real))

mae_train = mean_absolute_error(y_train_real, y_train_pred_real)
mae_test = mean_absolute_error(y_test_real, y_test_pred_real)

# **打印结果**
print(f"训练集 RMSE: {rmse_train:.2f}, MAE: {mae_train:.2f}")
print(f"测试集 RMSE: {rmse_test:.2f}, MAE: {mae_test:.2f}")

# 定义6折交叉验证
kf = KFold(n_splits=6, shuffle=True, random_state=42)

# 存储评估指标
rmse_scores = []
mae_scores = []

for train_index, val_index in kf.split(X):
    # **划分训练集和验证集**
    X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
    y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()

    # **获取验证集的建筑面积（确保是原始面积）**
    area_val = np.exp(data.loc[X_val.index, 'log_建筑面积'])  

    # 目标编码（区域变量）
    region_mean_map = X_train.join(y_train).groupby('区域')['log_建筑单价'].mean()
    X_train['区域_encoded'] = X_train['区域'].map(region_mean_map)
    X_val['区域_encoded'] = X_val['区域'].map(region_mean_map)

    # 处理缺失值，避免警告
    X_val['区域_encoded'] = X_val['区域_encoded'].fillna(y_train.mean())

    # **标准化**
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # **训练 Ridge 回归**
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_train_scaled, y_train)

    # **预测**
    y_val_pred = ridge_model.predict(X_val_scaled)

    # **反变换 + 乘以建筑面积计算总价**
    y_val_pred_price = np.exp(y_val_pred) * area_val  # 预测总价
    y_val_actual_price = np.exp(y_val) * area_val  # 真实总价

    # **计算 RMSE 和 MAE**
    rmse = np.sqrt(mean_squared_error(y_val_actual_price, y_val_pred_price))
    mae = mean_absolute_error(y_val_actual_price, y_val_pred_price)

    rmse_scores.append(rmse)
    mae_scores.append(mae)

# **计算交叉验证的平均 RMSE 和 MAE**
mean_rmse = np.mean(rmse_scores)
mean_mae = np.mean(mae_scores)

# **打印结果**
print(f"6折交叉验证 平均 RMSE（总价）: {mean_rmse:.2f}")
print(f"6折交叉验证 平均 MAE（总价）: {mean_mae:.2f}")


数据读取成功！
Ridge回归均方误差: 0.07702843765737578
Ridge回归系数: [ 0.00366802  0.01503882  0.05105828 -0.0093873   0.00508669  0.02302044
  0.02736783 -0.01619167  0.02769776  0.01863063  0.00466596  0.06616343
 -0.02200755 -0.00111304  0.00182642  0.00989234  0.00652113  0.00264424
  0.01410601 -0.01294061 -0.00718224 -0.00376223  0.01099594  0.00153672
  0.72303236]
预测结果已保存到 C:\Users\HP\Desktop\mid_project\预测结果_ridge_04.csv
训练集 RMSE: 1070426.00, MAE: 430319.77
测试集 RMSE: 1038320.98, MAE: 426247.06
6折交叉验证 平均 RMSE（总价）: 1070193.92
6折交叉验证 平均 MAE（总价）: 434164.58
